Creating Spark Session and configuring CPU and Memory resources for this Spark Session. 
Read all the downloaded Parquet files in a Dataframe (df) from "parquet_files" folder. 
Infer schema from parquet files automatically and create a Table like temporary view "ccnews".
Now we can query data from "ccnews" just like any other tabe using SQL queries.

In [1]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[4]') \
    .appName('predictive_analysis') \
    .config('spark.executor.memory', '10gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

df = spark.read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .option("compression", "snappy") \
    .parquet("../parquet_files/")

df.createOrReplaceTempView("ccnews")

Create a DataFrame from the result of Spark SQL Query
Convert spark dataframe to Pandas Data Frame for visualizations
Convert Pandas dataframe to Spark Data Frame

In [2]:
#query_df = spark.sql("select count(*) from ccnews")
#pandas_df = query_df.select("*").toPandas()
#spark_df = spark.createDataFrame(pandas_df)

In [15]:
spark.sql("describe ccnews").show()

+--------------+---------+-------+
|      col_name|data_type|comment|
+--------------+---------+-------+
| requested_url|   string|   NULL|
|    plain_text|   string|   NULL|
|published_date|   string|   NULL|
|         title|   string|   NULL|
|          tags|   string|   NULL|
|    categories|   string|   NULL|
|        author|   string|   NULL|
|      sitename|   string|   NULL|
|     image_url|   string|   NULL|
|      language|   string|   NULL|
|language_score|   double|   NULL|
| responded_url|   string|   NULL|
|     publisher|   string|   NULL|
|     warc_path|   string|   NULL|
|    crawl_date|   string|   NULL|
+--------------+---------+-------+



In [50]:
spark.sql("select publisher, count(1) as cp, \
ROW_NUMBER() OVER (PARTITION BY language ORDER BY publisher) as rank \
FROM ccnews GROUP BY language, publisher").show()

+------------------+-----+----+
|         publisher|   cp|rank|
+------------------+-----+----+
|     citizen.co.za|    2|   1|
|     kwevoel.co.za|    2|   2|
|     proagri.co.za|   50|   3|
|am.vse-o-tattoo.ru|    1|   1|
|            dw.com|   75|   2|
|  einpresswire.com|    3|   3|
|      4algeria.com|   28|   1|
|      agrimaroc.ma|    7|   2|
|       aitnews.com|   45|   3|
| akhbarlibya24.net| 2827|   4|
|   alahalygate.com| 2010|   5|
|         alalam.ir| 7399|   6|
|     alarabiya.net|11658|   7|
|   albaathmedia.sy|  687|   8|
|  albawabhnews.com|34997|   9|
|        albayan.ae|18031|  10|
|  albiladdaily.com| 1724|  11|
|         aleqt.com|  768|  12|
|       aletihad.ae|12494|  13|
|        alghad.com| 9088|  14|
+------------------+-----+----+
only showing top 20 rows

